In [1]:
!pip install -i https://test.pypi.org/simple/ supportlib



Looking in indexes: https://test.pypi.org/simple/


In [0]:
import supportlib.gettingdata as getdata
getdata.kaggle()

In [0]:
!pip install -q kaggle

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
getdata.zipextract('/content/glove6b300dtxt.zip')
#!pwd

In [0]:
!kaggle datasets download -d thanakomsn/glove6b300dtxt

In [0]:
!git clone https://github.com/ipmob/NITD-machine-Learning-challange

In [0]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import re
import keras

import nltk 
nltk.download('punkt')

nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))

In [0]:
def preprocess_text(text):
    
    #Lowering case the text
    text = text.lower()
    
    #Removing numbers
    text = re.sub(r'\d+', '', text)
    
    #Removing punctuations
    import string
    from nltk.tokenize import word_tokenize
    text = text.translate(str.maketrans('', '', string.punctuation))

    #Removing stop words
    
    tokens = word_tokenize(text)
    text = ' '.join([ i for i in tokens if not i in stopwords])
    
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    return text

In [0]:
data = pd.read_csv('/content/NITD-machine-Learning-challange/data/train_file.csv')
test = pd.read_csv('/content/NITD-machine-Learning-challange/data/test_file.csv')
sample = pd.read_csv('/content/NITD-machine-Learning-challange/data/results_file.csv')

data.loc[data.Subjects.isnull(), 'Subjects'] = ''
test.loc[test.Subjects.isnull(), 'Subjects'] = ''

data['combined'] = data.Subjects.str.cat(' ' + data.Title)
test['combined'] = test.Subjects.str.cat(' ' + test.Title)

dataMatType = data.MaterialType
data.drop(labels = ['MaterialType'], axis = 1, inplace = True)

data.combined = data.combined.apply(preprocess_text)
test.combined = test.combined.apply(preprocess_text)
'''
data.Subjects = data.Subjects.apply(preprocess_text)
data.Title = data.Title.apply(preprocess_text)
test.Subjects = test.Subjects.apply(preprocess_text)
test.Title = test.Title.apply(preprocess_text)
'''
flag = -1
labelDict = {}

for key in dataMatType.value_counts().keys():
  flag += 1
  labelDict[key] = flag
  
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 10                             

max_words = 1000                                    

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data.combined)

dataSequences = tokenizer.texts_to_sequences(data.combined)
testSequences = tokenizer.texts_to_sequences(test.combined)

data_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(data_word_index))

In [0]:
from keras import preprocessing
x_train = preprocessing.sequence.pad_sequences(dataSequences, maxlen = 25)
x_test = preprocessing.sequence.pad_sequences(testSequences, maxlen = 25)

train_label = dataMatType.map(labelDict)

from keras.utils import to_categorical
labelDataBinary = to_categorical(train_label)

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(dataMatType), dataMatType)

In [0]:
import os
glove_dir = '/content'

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.300d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

embedding_dim = 300

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in data_word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [0]:
reverseLabelDict = {}

for key, value in labelDict.items():
  reverseLabelDict[value] = key
def predict(model, x_test):
  predictions = model.predict(x_test)
  predictions = pd.Series([np.argmax(i) for i in predictions], index = test.index)
  predictions = predictions.map(reverseLabelDict)
  return predictions

In [0]:
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, SimpleRNN, Conv1D, MaxPooling1D
from keras.layers import Dropout
from keras.layers import Dense

model1 = Sequential()
model1.add(Embedding(1000, 1000, input_length=25))

'''model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
'''


model1.add(Conv1D(128, 3,  activation='relu'))
model1.add(Dropout(0.3))

model1.add(LSTM(400, recurrent_dropout = 0.6, return_sequences = True))
model1.add(Dropout(0.6))

model1.add(LSTM(400, recurrent_dropout = 0.2))
model1.add(Dropout(0.2))

model1.add(Dense(8, activation='softmax'))

model1.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model1.fit(x_train, labelDataBinary,
                    epochs=16,
                    validation_split=0.2,
                    class_weight = class_weights)

model1.save('/content/m1.h5')
#0.86623 

predic1 = pd.Series(predict(model1, x_test))
submission = pd.concat([test.ID.astype(np.int), predic1], axis = 1)
submission.columns = ['ID' , 'MaterialType']
submission.to_csv('/content/submition_model1.csv', index = False, header = True)  


In [0]:
from keras.layers import CuDNNLSTM,CuDNNGRU
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, SimpleRNN, Conv1D, MaxPooling1D
from keras.layers import Dropout
from keras.layers import Dense
from keras import optimizers,callbacks
import keras
model2 = Sequential()
model2.add(Embedding(1000, 1000 ,input_length=25))



model2.add(Conv1D(256, 5, activation='elu'))
model2.add(Dropout(0.6))
model2.add(Conv1D(512, 5, activation='elu'))
model2.add(Dropout(0.6))
model2.add(Conv1D(1024, 5, activation='elu'))
model2.add(Dropout(0.6))

model2.add(Flatten())

model2.add(Dense(400, activation='elu'))
model2.add(Dense(100, activation='elu'))
model2.add(Dense(8, activation='softmax'))

#adam = optimizers.adam(lr = 0.0005)
adam = optimizers.adam(lr = 0.0026)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)
model2.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model2.fit(x_train, labelDataBinary,
                    epochs=16,
                    validation_split=0.2,
                    class_weight = class_weights,
                   callbacks=[reduce_lr])

model2.save('/content/m2.h5')
#0.85621
predic2 = pd.Series(predict(model2, x_test))
submission = pd.concat([test.ID.astype(np.int), predic2], axis = 1)
submission.columns = ['ID' , 'MaterialType']
submission.to_csv('/content/submition_model2.csv', index = False, header = True)  


In [0]:
from keras.layers import CuDNNLSTM,CuDNNGRU
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, SimpleRNN, Conv1D, MaxPooling1D
from keras.layers import Dropout
from keras.layers import Dense
from keras import optimizers,callbacks
import keras
model3 = Sequential()
model3.add(Embedding(1000, 1000 ,input_length=25))


model3.add(CuDNNGRU(600, return_sequences = True))
model3.add(Dropout(0.6))

model3.add(CuDNNGRU(600, return_sequences = True))
model3.add(Dropout(0.6))

model3.add(CuDNNGRU(800))
model3.add(Dropout(0.7))

model3.add(Dense(200, activation='elu'))
model3.add(Dense(50, activation='elu'))
model3.add(Dense(8, activation='softmax'))

adam = optimizers.adam(lr = 0.001)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)
model3.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model3.fit(x_train, labelDataBinary,
                    epochs=16,
                    validation_split=0.02,
                    class_weight = class_weights,
                   callbacks=[reduce_lr])

model3.save('/content/m3.h5')
#0.87688 
predic3 = pd.Series(predict(model3, x_test))
submission = pd.concat([test.ID.astype(np.int), predic3], axis = 1)
submission.columns = ['ID' , 'MaterialType']
submission.to_csv('/content/submition_model3.csv', index = False, header = True)  


In [0]:
from keras.layers import CuDNNLSTM,CuDNNGRU
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, SimpleRNN, Conv1D, MaxPooling1D
from keras.layers import Dropout
from keras.layers import Dense
from keras import optimizers,callbacks
import keras
model4 = Sequential()
model4.add(Embedding(1000, 1000 ,input_length=25))


model4.add(CuDNNGRU(600, return_sequences = True))
model4.add(Dropout(0.7))

model4.add(CuDNNGRU(600))
model4.add(Dropout(0.5))

model4.add(Dense(100, activation='elu'))
model4.add(Dense(50, activation='elu'))
model4.add(Dense(8, activation='softmax'))

adam = optimizers.adam(lr = 0.001)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)
model4.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model4.fit(x_train, labelDataBinary,
                    epochs=16,
                    validation_split=0.2,
                    class_weight = class_weights,
                   callbacks=[reduce_lr])

model4.save('/content/m4.h5')
#0.87181 
predic4 = pd.Series(predict(model4, x_test))
submission = pd.concat([test.ID.astype(np.int), predic4], axis = 1)
submission.columns = ['ID' , 'MaterialType']
submission.to_csv('/content/submition_model4.csv', index = False, header = True)  


**Using Embedding Layer**

In [0]:
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, SimpleRNN, Conv1D, MaxPooling1D
from keras.layers import Dropout
from keras.layers import Dense

model5 = Sequential()
model5.add(Embedding(1000, 300, input_length=25, weights=[embedding_matrix], trainable=False))

model5.add(Conv1D(64, 5, activation='relu'))
model5.add(MaxPooling1D(pool_size=4))


model5.add(LSTM(1000, recurrent_dropout = 0.3))
model5.add(Dropout(0.35))
model5.add(Dense(8, activation='softmax'))

model5.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model5.fit(x_train, labelDataBinary,
                    epochs=16,
                    batch_size=128,
                    validation_split=0.2,
                    class_weight = class_weights)


model5.save('/content/m5.h5')
predic5 = pd.Series(predict(model5, x_test))
submission = pd.concat([test.ID.astype(np.int), predic5], axis = 1)
submission.columns = ['ID' , 'MaterialType']
submission.to_csv('/content/submition_model5.csv', index = False, header = True)  


In [0]:
from keras.layers import CuDNNLSTM,CuDNNGRU
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, SimpleRNN, Conv1D, MaxPooling1D
from keras.layers import Dropout
from keras.layers import Dense
from keras import optimizers,callbacks
import keras
model6 = Sequential()
model6.add(Embedding(1000, 1000 ,input_length=25))


model6.add(CuDNNGRU(600, return_sequences = True))
#model.add(Dropout(0.6))
model6.add(Dropout(0.7))

model6.add(CuDNNGRU(500))
model6.add(Dropout(0.6))

model6.add(Dense(100, activation='elu'))
model6.add(Dense(50, activation='elu'))
model6.add(Dense(8, activation='softmax'))

adam = optimizers.adam(lr = 0.001)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)
model6.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model6.fit(x_train, labelDataBinary,
                    epochs=16,
                    validation_split=0.2,
                    class_weight = class_weights,
                   callbacks=[reduce_lr])

model6.save('/content/model6.1.h5')
#v1_0.87181 
#0.87181 
#0.87695
predic6 = pd.Series(predict(model6, x_test))
submission = pd.concat([test.ID.astype(np.int), predic6], axis = 1)
submission.columns = ['ID' , 'MaterialType']
submission.to_csv('/content/submition_model6.csv', index = False, header = True)  


In [0]:
"""
reverseLabelDict = {}

for key, value in labelDict.items():
  reverseLabelDict[value] = key
  
predictions = model.predict(x_test)
predictions = pd.Series([np.argmax(i) for i in predictions], index = test.index)
predictions = predictions.map(reverseLabelDict)

submission = pd.concat([test.ID.astype(np.int), predictions], axis = 1)
submission.columns = ['ID' , 'MaterialType']
"""

In [0]:
 # submission.to_csv('/content/submition_3.1_05_100_50_8.csv', index = False, header = True)

In [0]:
""""reverseLabelDict = {}

for key, value in labelDict.items():
  reverseLabelDict[value] = key
def predict(model, x_test):
  predictions = model.predict(x_test)
  predictions = pd.Series([np.argmax(i) for i in predictions], index = test.index)
  predictions = predictions.map(reverseLabelDict)
  submission = pd.concat([test.ID.astype(np.int), predictions], axis = 1)
  submission.columns = ['ID' , 'MaterialType']
  return predictions"""

In [0]:
num_model = 6
predic1 = pd.Series(predict(model1, x_test))
predic2 = pd.Series(predict(model2, x_test))
predic3 = pd.Series(predict(model3, x_test))
predic4 = pd.Series(predict(model4, x_test))
predic5 = pd.Series(predict(model5, x_test))
predic6 = pd.Series(predict(model6, x_test))

In [0]:
"""total = pd.DataFrame(pd.concat([predic1, predic2, predic3, predic4, predic5,predic6], axis = 1))
submissions = total.mode(axis = 1)[0]
submission = pd.concat([test.ID.astype(np.int), predictions], axis = 1)
submission.columns = ['ID' , 'MaterialType']
submission.to_csv('sub-agg.csv', index = False, header = True)"""

In [0]:
total = pd.DataFrame(pd.concat([predic1, predic2, predic3, predic4, predic5,predic6], axis = 1))
submissions = total.mode(axis = 1)[0]
#submissions = pd.concat([test.ID.astype(np.int), predictions], axis = 1)
submissions.columns = ['ID' , 'MaterialType']
submissions.to_csv('/content/sub-final.csv', index = False, header = True)